In [ ]:
# "Using Deep Learning to Classify Airliner Flight Profiles for Post-Flight Analysis"
> "Capstone Project for Master of Science in Data Science University of Wisconsin"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [4]:
import pandas as pd

In [2]:
1+1

2